In [ ]:
import ngram
import os
import preprocess

indir_pre = os.getcwd() + "/"
outdir_pre = os.getcwd() + "/"

def random_sentence_ngram(n = 2, sent_pre = "I have", topic = "autos"):
    # TODO: lili: create ngram class for every topic, and implemente sentence generation for each topic
    indir = indir_pre + "data/classification_task/{}/train_docs".format(topic)
    content = preprocess.preprocess(indir)
    ngrams = ngram.ngram_Generator()
    
    for k in xrange(1, n + 1):
        print "\n\n[{}-gram]\n".format(k)

        print "Empty sentence"
        for i in xrange(3):
            print "[{}]  ".format(i + 1) + ngrams.generate_sentence(k, content)

        print "\nWith incomplete sentence: " + "\"{}\"".format(sent_pre)
        for i in xrange(3):
            print "[{}]  ".format(i + 1) + ngrams.generate_sentence(k, content, sent_pre)

def main():
    random_sentence_ngram(topic="atheism")




if __name__ == "__main__":
    main()


In [1]:
import ngram
import os
import preprocess

indir_pre = os.getcwd() + "/"
outdir_pre = os.getcwd() + "/"

def random_sentence_ngram(n = 2, sent_pre = "I have", topic = "autos"):
    indir = indir_pre + "data/classification_task/{}/train_docs".format(topic)
    content = preprocess.preprocess_dir(indir)
    ngrams = ngram.ngram()

    for k in xrange(1, n + 1):
        print "\n\n[{}-gram]\n".format(k)

        print "Empty sentence"
        for i in xrange(3):
            print "[{}]  ".format(i + 1) + ngrams.generate_sentence(k, content)

        print "\nWith incomplete sentence: " + "\"{}\"".format(sent_pre)
        for i in xrange(3):
            print "[{}]  ".format(i + 1) + ngrams.generate_sentence(k, content, sent_pre)


def topic_classification():
    pass


def spell_checker():
    pass

def main():
    random_sentence_ngram(topic="atheism")




if __name__ == "__main__":
    main()




[1-gram]

Empty sentence
[1]  We , in a system merely as for after country not gregg an might it with I
[2]  Of evidence . country since
[3]  The prison mikec is . . not I salesman mathew or discussed were is encouraged toleration those deleted aren of independence any individuals he us the the what , american he sort on nothing . you re when is we you jaeger am you morals fault you you will people very terms is ones been 90650 starting , these I it matthew about to , fight gregg you but kill see possible this . system higher mike I realm saw 3 this decide so there

With incomplete sentence: "I have"
[1]  I have of east bifurcations if well the young and re . do couple sexual it map societally do that
[2]  I have none of
[3]  I have criminal when to maintain beginning trust who too


[2-gram]

Empty sentence
[1]  Johnson in country being absorbed into reality .
[2]  He said to prevent this smart people to say that there s not including a prophecy for anyone respect , I m sure this wa